In [1]:

from __future__ import print_function

import numpy as np
import gensim
import string

from keras.callbacks import LambdaCallback
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.utils.data_utils import get_file

print('\nFetching the text...')
url = 'https://ota.bodleian.ox.ac.uk/repository/xmlui/bitstream/handle/20.500.12024/5730/5730.txt'
path = get_file('tresure_island.txt', origin=url)

print('\nPreparing the sentences...')
max_sentence_len = 20
with open(path) as file_:
  docs = file_.readlines()
sentences = [[word for word in doc.lower().translate(string.punctuation).split()[:max_sentence_len]] for doc in docs]
print('Num sentences:', len(sentences))
print(sentences)

print('\nTraining word2vec...')
word_model = gensim.models.Word2Vec(sentences, size=100, min_count=1, window=5, iter=100)
pretrained_weights = word_model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)
print('Checking similar words:')
for word in ['american', 'evening', 'treasure', 'captain']:
  most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in word_model.most_similar(word)[:8])
  print('  %s -> %s' % (word, most_similar))

def word2idx(word):
  return word_model.wv.vocab[word].index
def idx2word(idx):
  return word_model.wv.index2word[idx]

print('\nPreparing the data for LSTM...')
train_x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
train_y = np.zeros([len(sentences)], dtype=np.int32)
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    train_x[i, t] = word2idx(word)
  train_y[i] = word2idx(sentence[-1])
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)

print('\nTraining LSTM...')
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))
model.add(LSTM(units=emdedding_size))
model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

def generate_next(text, num_generated=10):
  word_idxs = [word2idx(word) for word in text.lower().split()]
  for i in range(num_generated):
    prediction = model.predict(x=np.array(word_idxs))
    idx = sample(prediction[-1], temperature=0.7)
    word_idxs.append(idx)
  return ' '.join(idx2word(idx) for idx in word_idxs)

def on_epoch_end(epoch, _):
  print('\nGenerating text after epoch: %d' % epoch)
  texts = [
    'one',
    'classic',
    'i remember',
    'a',
  ]
  for text in texts:
    sample = generate_next(text)
    print('%s... -> %s' % (text, sample))

model.fit(train_x, train_y,
          batch_size=128,
          epochs=20,
          callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])


Fetching the text...
360448/358061 [==============================] - 0s 0us/step

Preparing the sentences...
Num sentences: 1419
[['treasure', 'island+'], ['to', 's.l.o.,', 'an', 'american', 'gentleman', 'in', 'accordance', 'with', 'whose', 'classic', 'taste', 'the', 'following', 'narrative', 'has', 'been', 'designed,', 'it', 'is', 'now,'], ['to', 'the', 'hesitating', 'purchaser+'], ['if', 'sailor', 'tales', 'to', 'sailor', 'tunes,+'], ['storm', 'and', 'adventure,', 'heat', 'and', 'cold,+'], ['if', 'schooners,', 'islands,', 'and', 'maroons,+'], ['and', 'buccaneers,', 'and', 'buried', 'gold,+'], ['and', 'all', 'the', 'old', 'romance,', 'retold+'], ['exactly', 'in', 'the', 'ancient', 'way,+'], ['can', 'please,', 'as', 'me', 'they', 'pleased', 'of', 'old,+'], ['the', 'wiser', 'youngsters', 'of', 'today:+'], ['part', 'one—the', 'old', 'buccaneer+'], ['the', 'old', 'sea-dog', 'at', 'the', 'admiral', 'benbow+'], ['squire', 'trelawney,', 'dr.', 'livesey,', 'and', 'the', 'rest', 'of', 'these

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


Result embedding shape: (5209, 100)
Checking similar words:
  american -> s.l.o., (0.97), accordance (0.95), away.+ (0.92), instant (0.91), treasure-hunt—flint's (0.89), ebb-tide (0.88), youngsters (0.88), humility, (0.88)
  evening -> idle, (0.80), trim (0.79), concern, (0.77), sealed (0.76), favour. (0.76), minds, (0.76), merry's (0.75), blundered (0.75)
  treasure -> rear, (0.82), quiet (0.82), western (0.80), island. (0.80), report (0.80), clearing. (0.79), balls (0.79), inside (0.77)
  captain -> "load (0.59), "well?" (0.58), fellow (0.54), doctor (0.53), smollett (0.53), chart (0.53), door. (0.50), return, (0.50)

Preparing the data for LSTM...
train_x shape: (1419, 20)
train_y shape: (1419,)

Training LSTM...
Epoch 1/20
12/12 [==============================] - 31s 18ms/step - loss: 8.5190

Generating text after epoch: 0
one... -> one me. son. steel. visibly way, coming unarmed, escape berth five
classic... -> classic district, care," "you'll minds, strike see "well?" tore breakf